# Volumeter

TODO
- measurementCycle(loweringSpeed)
- calibrateHolder()
- syncMeasurementData() # Needs to incorporate some sort of interpolation to sync based on immersion depth insted of samples. Maybe the volume should be calculated in this function.
- ~calculateVolume()
- visualize(), matplotlib, plotly, something else?
- exportCSV()
- Mittaustelineen valinta-UI widgetseillä
- Nippusiteiden paikkojen valinta widgetseillä

Pandas resampling
https://stackoverflow.com/questions/26517125/combine-two-pandas-dataframes-resample-on-one-time-column-interpolate
https://stackoverflow.com/questions/20941973/python-pandas-interpolate-with-new-x-axis


In [2]:
import serial
import pandas as pd
import numpy as np
import time
import os
from math import ceil
from IPython.core.display import clear_output

In [1]:
global loadCellConstants
global mcSerial
global sensorSerial
global homingDone

def readLoadCellCalibration():
    '''Loads the load cell calibration values from a CSV file called loadCellCalibration.csv.'''
    cal = np.genfromtxt('loadCellCalibration.csv', delimiter=',')
    return cal

def loadCellToGrams(value):
    '''Converts a load cell reading to grams based on the calibration values.'''
    return value*loadCellConstants[0] + loadCellConstants[1]

def initMotionControl(motionControlPort='/dev/ttyUSB1'):
    '''Initializes the motion control serial port.
    
    Uses a default of /dev/ttyUSB1 but the port can be given as an argument.
    '''
    global mcSerial
    global homingDone
    mcSerial = serial.Serial(port=motionControlPort, baudrate=115200)
    homingDone = False

def initSensors(sensorPort='/dev/ttyUSB0'):
    '''Initializes the sensor serial port.
    
    Uses a default of /dev/ttyUSB0 but the port can be given as an argument.
    '''
    global sensorSerial
    global loadCellConstants
    loadCellConstants = readLoadCellCalibration()
    
    sensorSerial = serial.Serial(port=sensorPort, baudrate=115200)

def closeSerial(serial):
    '''Closes the serial port that was given as an argument'''
    serial.close()

def sendMotionCommand(cmd):
    '''Sends the given gcode command to the motion control serial port.'''
    mcSerial.write('{0}\n'.format(cmd).encode('utf-8'))
    mcSerial.flushInput()
    grbl_out = mcSerial.readline()
    return grbl_out.decode()

def motionFinished(wait=False):
    '''Returns True if the motion control system is idle
    
    Giving wait=True as an argument waits until the motion control system becomes idle and returns True.
    '''
    finished = False
    if(wait):
        while(not finished):
            time.sleep(0.5)
            grbl_out = sendMotionCommand('?')
            finished = 'Idle' in str(grbl_out)
        return True
    else:
        grbl_out = sendMotionCommand(serial, '?')
        return 'Idle' in str(grbl_out)

def homingCycle():
    '''Homes the motion control system axes.'''
    
    global homingDone
    
    cycle = ['?', 'G91', 'G38.2Z2500F2000', 'G1Z-20F1000',
             'G38.2X-560Z-560F600', 'G1X10Z10F600', 'G10L20P1X0Y0Z0',
            'G90', 'G1Z-1650F3000']
    for c in cycle:
        sendMotionCommand(c)
    motionFinished(wait=True)
    homingDone = True

def recordSensors(numSamples):
    '''Records the sensors for the given number of samples and return the in a pandas DataFrame.'''
    sensorSerial.flushInput()
    samplesRead = 0
    df = pd.DataFrame(columns=['time', 'loadCell', 'level'], dtype='float')
    while(samplesRead < numSamples):
        try:
            data = np.array(sensorSerial.readline().decode().rstrip().split(','))
            data = data.astype(np.float)
            df.loc[len(df)] = data
            samplesRead += 1
        except:
            pass
    # Will fixing this inplace=False issue break anything?
    # df = df.set_index(['time'])
    return df

def resampleDF(df, resolution=0.5):
    '''Resamples the data to 0.5 resolution by default on the index axis.'''
    # TODO: NOT TESTED
    idx = pd.Index(np.arange(0, ceil(df.index.values.max()), resolution))
    newIdx = idx.union(df.index)
    dfR = df.reindex(newIdx).interpolate(method='index')
    dfR = dfR.reindex(idx)
    return dfR

def meanLoadCell(numSamples):
    '''Returns the mean load cell value of given number of samples'''
    data = recordSensors(numSamples)
    return data['loadCell'].mean()

def meanWeight(numSamples):
    '''Returns the mean weight in grams of given number of samples'''
    data = recordSensors(numSamples)
    mean = data['loadCell'].mean()
    return loadCellToGrams(mean)

def calibrateLoadCell(numWeights=1):
    # TODO Interpolate depth data to 0,5mm resolution for example
    '''Runs a load cell calibration wizard.
    
    numWeights argument can be used to tell how many calibration weights you want to use.
    The calibration values are saved to loadCellCalibration.csv if the user accepts the new calibration values.
    '''
    grams = []
    values = []
    print("Load Cell Calibration")
    print("Leave the specimen holder empty")
    input("Hit enter when the specimen holder is stationary")
    print("Wait for 5 seconds...")
    grams.append(0.0)
    values.append(meanLoadCell(80*5))
    while(numWeights > 0):
        numWeights -= 1
        while(True):
            try:
                calWeight = input("Mass of the calibration weight in kg: ").rstrip().replace(',', '.')
                grams.append(float(calWeight)*1000.0)
                break
            except:
                print("Insuitable mass value")

        print("Add the calibratrion weight to the specimen holder")
        input("Hit enter when the specimen holder is stationary")
        print("Wait for 5 seconds...")
        values.append(meanLoadCellmeanWeight(80*5))
    calibrationConstants = np.polyfit(values, grams, 1)
    print("Calibration: y = {:.2f}x+{:.2f}".format(calibrationConstants[0], calibrationConstants[1]))
    save = input("Save calibration (y/n)")
    if('y' in save.lower()):
        np.savetxt('loadCellCalibration.csv', calibrationConstants, delimiter=',')
        print('Calibration saved')
    else:
        print('Calibration cancelled')
        
def calibrateHolder(numTimes=3):
    '''Runs a specimen holder calibration cycle and asks the user for a name of the calibration.

    Calibration values are saved to holderCalibrations.csv file whose column specifies the name of the holder.
    '''
    
    loweringSpeed = 2000 # mm/min
    topHeigth = 300
    bottomHeigth = -850
    smoothing = 10
    
    if(not homingDone):
        print("Motion control system is not homed. Please run the homingCycle() first.")
        return None

    motionFinished(wait=True)

    # Move on top of the water tank
    movementsToTopOfTank = ['?', 'G90', 'G1X0Z-1640F6000','G1X0Z-50F4000', 'G1X540Z540F3000','G1X540Z{}F2000'.format(topHeigth)]
    for c in movementsToTopOfTank:
        sendMotionCommand(c)
    motionFinished(wait=True)
    
    # Dip the holder to the water to make it wet
    #sendMotionCommand('G1X540Z{}F2000'.format(bottomHeigth))
    #sendMotionCommand('G1X540Z{0}F2000'.format(topHeigth))
    
    df = pd.DataFrame(columns=['depth', 'force'], dtype='float')
    
    while(numTimes > 0):
        numTimes -= 1
        # Wait for the movement to stabilize
        time.sleep(5)
        sendMotionCommand('G1X540Z{}F2000'.format(bottomHeigth))
        time.sleep((loweringSpeed/60)/20) # 20mm/s^2 acceleration, start recording data after the acceleration
        data = recordSensors(86*((topHeigth-bottomHeigth)/(loweringSpeed/60)))
        motionFinished(wait=True)
        time.sleep(1)
        sendMotionCommand('G1X540Z{0}F2000'.format(topHeigth))
        
        # TODO This should be its own function?
        df['depth'] = ((data['time']-data['time'].iloc[int(smoothing/2)])*loweringSpeed/60)/1000.0 + data['level'] - data['level'].iloc[int(smoothing/2)]
        df['force'] = data['loadCell'].apply(loadCellToGrams)
        df['force'] = df['force'].rolling(10, center=True).median()
        df = df.dropna()
        df = df.set_index(['depth'])
        df = resampleDF(df)
        print(df.head(30))
        print('---')
        print(df.tail())
        
        motionFinished(wait=True)
    
    for c in reversed(movementsToTopOfTank):
        sendMotionCommand(c)
    motionFinished(wait=True)                        
        
def selectSpecimenHolder():
    '''Lists the names of the available specimen holder calibrations and asks the user to select one of them.'''

def measurementCycle():
    '''Runs the measurement cycle.'''
    if(not homingDone):
        print("Motion control system is not homed. Please run the homingCycle() first.")
        return None

    motionFinished(wait=True)
    
    #Record the mean weight of the specimen
    meanWeight = meanWeight(200)
    print('Specimen weight: {:.2f}'.format(meanWeight))

    
        

In [9]:
initMotionControl()
finished = motionFinished(wait=True)
homingCycle()
#closeSerial(mcSerial)

In [33]:
initSensors()
#data = recordSensors(10)
for i in range(10):
    mw = meanWeight(20)
    clear_output(wait=True)
    print('{:.2f}'.format(mw))
#mw = meanWeight(10)
#calibrateLoadCell(numWeights = 2)
#print(mw)

194.03


In [53]:
calibrateHolder(1)

        depth       force
5    0.000000  202.765332
6    0.366667  203.458784
7    0.766667  206.802214
8    1.133333  206.802214
9    1.533333  203.458784
10   1.933333  206.059230
11   2.300000  204.449430
12   2.700000  203.755978
13   3.100000  203.755978
14   3.466667  203.755978
15   3.833333  203.755978
16   4.266667  203.755978
17   4.633333  201.130767
18   5.033333  203.755978
19   5.400000  206.628851
20   5.800000  205.885867
21   6.200000  206.331657
22   6.566667  207.470900
23   6.966667  207.470900
24   8.366667  207.470900
25   8.733333  206.331657
26   9.133333  206.331657
27   9.533333  206.331657
28   9.900000  204.251301
29  10.266667  198.059765
30  10.666667  198.059765
31  11.066667  198.059765
32  11.433333  196.326134
33  11.833333  196.326134
34  12.233333  203.260655
---
            depth       force
2958  1154.333333 -423.620025
2959  1154.733333 -423.620025
2960  1155.133333 -424.808800
2961  1155.500000 -424.833566
2962  1155.866667 -426.319535


In [54]:
closeSerial(mcSerial)
closeSerial(sensorSerial)

In [ ]:
os.system('sudo shutdown.sh')